<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

**Признаки**
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

**Целевой признак**
- Exited — факт ухода клиента

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/datasets/Churn.csv')

In [ ]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В дальнейшем нам не понадобятся следующие столбцы: RowNumber, CustomerId, Surname

In [ ]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Посмотрим на наличие дубликатов

In [ ]:
data.duplicated().sum()

0

In [ ]:
data.isna().sum()

CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Можно заметить пропуски в одном из столбцов, поскольку пропусков очень мало по соотношению с данными, то можно их удалить

In [ ]:
data = data.dropna(subset = ["Tenure"], axis = 0)

Подготовим данные с помощью ОНЕ, чтобы не словить дамми ловушку

In [ ]:
data = pd.get_dummies(data, drop_first=True)

Вывод: явных дубликатов в данных не было обнаружено, но были пропущенные значения в столбце Tenure, где было принято решение удалить их. Также были удаленыы столбцы, которые не понадобились бы в дальнейшем RowNumber', CustomerId, Surname

**Изменено**

## Исследование задачи

Разделим выборки на тренировочную, валидационную и обучающую выборку

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.4, random_state=12345)
features = data_test.drop('Exited', axis=1)
target = data_test['Exited']

In [ ]:
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)
features_train = data_train.drop('Exited', axis=1)
target_train = data_train['Exited']

**Масштабируемость для линейной регрессии**

Для начала зафиксируем численные признаки

In [ ]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

Масштабируем числ признаки обучающей выборки

In [ ]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9344,0.809075,-1.039327,-1.025995,0.554904,-0.908179,1,0,0.019508,0,0,0
3796,-1.152518,-1.227561,0.696524,0.480609,-0.908179,0,0,0.056167,0,0,1
7462,-0.398853,0.090079,1.385532,-1.237830,-0.908179,1,1,0.848738,0,0,1
1508,-0.749875,-0.286389,0.352020,-1.237830,0.809300,1,1,-0.894953,0,0,1
4478,-1.028628,-0.756975,-0.336987,-1.237830,0.809300,0,1,-1.284516,0,0,1


Масштабируем численные признаки валидационной выборки

In [ ]:
 features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

/tmp/ipykernel_31/3700386726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_valid[numeric] = scaler.transform(features_valid[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7445,-1.369326,0.560665,-0.336987,-1.237830,-0.908179,1,0,-0.086537,0,0,0
8620,1.232367,0.090079,1.041028,-1.237830,0.809300,0,1,-0.537457,0,0,0
1714,0.840048,0.560665,0.352020,1.231363,-0.908179,0,0,1.070393,1,0,1
5441,1.056856,-0.945210,-1.370498,0.951231,-0.908179,1,0,-0.576279,0,0,1
9001,0.406433,-0.662858,0.352020,0.767800,-0.908179,1,1,0.662068,0,1,1


Масштабируем численные признаки тестовой выборки

In [ ]:
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

/tmp/ipykernel_31/2409317702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[numeric] = scaler.transform(features_test[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
5170,1.707279,-0.756975,-0.336987,-1.237830,0.809300,1,1,0.718362,0,0,0
4180,-0.429826,-0.286389,1.730036,0.485538,0.809300,0,0,1.687305,1,0,1
7349,-0.171721,0.278313,0.352020,-0.269213,0.809300,0,1,0.824128,1,0,1
7469,0.385784,-0.380507,1.041028,0.464813,-0.908179,1,0,-1.118018,0,0,0
3467,-1.142194,0.278313,-1.370498,0.353837,0.809300,0,1,-0.107642,1,0,0


Проверка баланса классов

In [ ]:
target_train.value_counts(normalize = 1)

0    0.793546
1    0.206454
Name: Exited, dtype: float64

In [ ]:
target_valid.value_counts(normalize = 1)

0    0.792629
1    0.207371
Name: Exited, dtype: float64

Можно заметить, что ответов 0 - 80 %, а ответов 1 - 20%. На основании этого можно сделать вывод о том, что наблюдается дисбаланс

Дерево решений


In [ ]:
best_result = 0
depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    AUC = roc_auc_score(target_valid, probabilities_one_valid)
    if result > best_result:
        best_depth = depth
        best_result = result

print("Глубина дерева:", best_depth, "F1:", best_result,'AUC-ROC',AUC)

Глубина дерева: 7 F1: 0.5764331210191083 AUC-ROC 0.7566971065260089


Случайный лес

In [ ]:
best_result = 0
depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_train , target_train)
            predictions = model.predict(features_valid)
            result = f1_score(predictions, target_valid)
            probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
            AUC = roc_auc_score(target_valid, probabilities_one_valid)
            if result > best_result:
                best_result = result
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", best_result,'AUC-ROC',AUC,sample)

Глубина дерева: 16 Количество деревьев: 15 F1: 0.6026936026936026 AUC-ROC 0.8650933167911321 4


Логическая регрессия

In [ ]:
model = LogisticRegression()
model.fit(features_train , target_train)
predictions = model.predict(features_valid)
best_result = f1_score(predictions, target_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
AUC = roc_auc_score(target_valid, probabilities_one_valid)
print("F1:", best_result,'AUC-ROC',AUC)

F1: 0.30400000000000005 AUC-ROC 0.773663293800172


Лучшее значение f1 у модели случайный лес

## Борьба с дисбалансом

Увеличение выборки

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [ ]:
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, repeat=4)

In [ ]:
target_upsampled_train.value_counts(normalize = 1)

1    0.509964
0    0.490036
Name: Exited, dtype: float64

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

👍 Отлично, что проверяем баланс классов после преобразований.
</div>


In [ ]:
best_result = 0
depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_upsampled_train, target_upsampled_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    AUC = roc_auc_score(target_valid, probabilities_one_valid)
    if result > best_result:
        best_depth = depth
        best_result = result

print("Глубина дерева:", best_depth, "F1:", best_result,'AUC-ROC',AUC)

Глубина дерева: 5 F1: 0.5735449735449736 AUC-ROC 0.7295110785502994


In [ ]:
best_result = 0
depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_upsampled_train, target_upsampled_train)
            predictions = model.predict(features_valid)
            result = f1_score(predictions, target_valid)
            probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
            AUC = roc_auc_score(target_valid, probabilities_one_valid)
            if result > best_result:
                best_result = result
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", best_result,'AUC-ROC',AUC,sample)

In [ ]:
model = LogisticRegression()
model.fit(features_upsampled_train, target_upsampled_train)
predictions = model.predict(features_valid)
best_result = f1_score(predictions, target_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
AUC = roc_auc_score(target_valid, probabilities_one_valid)
print("F1:", best_result,'AUC-ROC',AUC)

Уменьшение выборки

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [ ]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.25)

In [ ]:
target_downsampled_train.value_counts(normalize = 1)

In [ ]:
best_result = 0
depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_downsampled_train, target_downsampled_train)
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    AUC = roc_auc_score(target_valid, probabilities_one_valid)
    if result > best_result:
        best_depth = depth
        best_result = result

print("Глубина дерева:", best_depth, "F1:", best_result,'AUC-ROC',AUC)

In [ ]:
best_result = 0
depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_downsampled_train, target_downsampled_train)
            predictions = model.predict(features_valid)
            result = f1_score(predictions, target_valid)
            probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
            AUC = roc_auc_score(target_valid, probabilities_one_valid)
            if result > best_result:
                best_result = result
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", best_result,'AUC-ROC',AUC,sample)

In [ ]:
model = LogisticRegression()
model.fit(features_downsampled_train, target_downsampled_train)
predictions = model.predict(features_valid)
best_result = f1_score(predictions, target_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
AUC = roc_auc_score(target_valid, probabilities_one_valid)
print("F1:", best_result,'AUC-ROC',AUC)

Благодаря сбаланнисрованности классов, F1 меры выше 0,59. Для тестирования лучше всего взять модель Случайного леса, так как у нее лучший результат F1 меры, с гиперпараметрами: max_depth = 18 ; n_estimators = 45; min_samples_leaf = 4; random_state = 1234

## Тестирование модели

In [ ]:
model = RandomForestClassifier(max_depth=18, n_estimators=45, min_samples_leaf=4, random_state=1234)
model.fit(features_upsampled_train, target_upsampled_train)
predictions = model.predict(features_test)
result = f1_score(predictions, target_test)
probabilities_one_valid = model.predict_proba(features_test)[:, 1]
AUC = roc_auc_score(target_test, probabilities_one_valid)

print("F1:",result,'AUC-ROC',AUC)

**Итоговый вывод**

Сравнив  модели дерево решений, случаный лес и линейную регрессию лучшего всего себя показала модель случайного леса с гиперпараметрами: max_depth = 18 ; n_estimators = 45; min_samples_leaf = 4; random_state = 1234.